In [ ]:
import math
import matplotlib.pyplot as mpl

# Times in seconds
def ActivityAtTime( StartingActivity, TimeElapsed, HalfLife ):
    return StartingActivity * ( 2.0 ** ( -TimeElapsed / HalfLife ) )

def F18ActivityAtTime( StartingActivity, TimeElapsed ):
    return ActivityAtTime( StartingActivity, TimeElapsed, 109.77*60.0 )

def Zr89ActivityAtTime( StartingActivity, TimeElapsed ):
    return ActivityAtTime( StartingActivity, TimeElapsed, 78.41*60.0*60.0 )

def ActivityPlot( StartingActivity, EndTime, ActivityMethod ):
    activityValues = []
    timeValues = []
    time = 0.0
    while time < EndTime:
        activityValues.append( ActivityMethod( StartingActivity, time ) )
        timeValues.append( time )
        time += 1.0
        
    mpl.plot( timeValues, activityValues )
    
ActivityPlot( 1E6, 200.0*60.0, F18ActivityAtTime )
ActivityPlot( 1E6, 200.0*60.0, Zr89ActivityAtTime )
mpl.show()

In [ ]:
import random

# Simulate poisson-distributed random decay times
# Based on example https://timeseriesreasoning.com/contents/poisson-process/
def DeltaT( DecayRate ):
    randUniform = random.random()
    return -math.log( 1.0 - randUniform ) / DecayRate

def ActivityTimeline( DecayRate, EndTime ):
    decayTimes = []
    time = 0.0
    while time < EndTime:
        time += DeltaT( DecayRate )
        if time < EndTime:
            decayTimes.append( time )
            
    return decayTimes
            
def TimelinesPlot( Timelines ):
    for i, decayTimes in enumerate( Timelines ):
        dummyY = []
        for time in decayTimes:
            dummyY.append( i )
            
        mpl.scatter( decayTimes, dummyY )

TimelinesPlot( [ ActivityTimeline( 2, 20 ), ActivityTimeline( 1, 20 ) ] )
mpl.show()

In [ ]:
import sys
from matplotlib.patches import Rectangle

def FindCoincidences( Timelines, TimeWindow ):
    coincidences = []
    coincidenceTimes = []
    
    # Find the first event in each timeline
    nextTimes = []
    nextIndices = []
    for decayTimes in Timelines:
        if len( decayTimes ) == 0:
            nextTimes.append( sys.float_info.max )
        else:
            nextTimes.append( decayTimes[0] )
        nextIndices.append( 0 )
    
    unfinishedTimeline = True
    while unfinishedTimeline:

        # Start the window at the first unprocessed event
        time = min( nextTimes )
        #print( nextTimes, " min = ", time )
    
        # Find all events in the window
        coincidence = []
        for timelineIndex, nextTime in enumerate( nextTimes ):
            #print( "testing", timelineIndex, nextTime, " against ", time )
            if nextTime >= time and nextTime <= time + TimeWindow \
                and nextTime < sys.float_info.max - TimeWindow: # to catch the dummy values
                #print( "coincidence!" )
                coincidence.append( timelineIndex )
                
        # Catch a no-coincidence (including self) because we've ended up all infinities
        if len( coincidence ) == 0:
            break
    
        # Update to the next time for anything in the coincidence window
        unfinishedTimeline = False
        finalCoincidence = []
        for timelineIndex in coincidence:
            
            # Find the first time that's outside the current window
            nextIndex = nextIndices[ timelineIndex ]
            nextTime = Timelines[ timelineIndex ][ nextIndex ]
            while nextTime <= time + TimeWindow:
                
                # Add entries if there is coincidence within the window for a single timeline
                finalCoincidence.append( timelineIndex )
                nextIndex += 1
            
                # Test for reaching the end of the timeline
                if nextIndex >= len( Timelines[ timelineIndex ] ):
                    nextTimes[ timelineIndex ] = sys.float_info.max
                    nextIndices[ timelineIndex ] = -1
                    break
                else:
                    nextTime = Timelines[ timelineIndex ][ nextIndex ]
                    nextTimes[ timelineIndex ] = nextTime
                    nextIndices[ timelineIndex ] = nextIndex
                    unfinishedTimeline = True
        
        # Need to search for dummy indices since we don't update everything each round
        for nextIndex in nextIndices:
            if nextIndex > -1:
                unfinishedTimeline = True
                break
                
        coincidences.append( finalCoincidence )
        coincidenceTimes.append( time )
    
    return coincidences, coincidenceTimes

def CoincidenceBoxes( Coincidences, CoincidenceTimes, TimeWindow ):
    
    for i, coincidence in enumerate( Coincidences ):
        time = CoincidenceTimes[ i ]
        y = min( coincidence )
        height = max( coincidence ) - y
        y -= 0.1
        height += 0.2
        mpl.gca().add_patch( Rectangle( (time, y), TimeWindow, height, \
                                        linewidth=1,edgecolor='r',facecolor='none') )
        
exampleTimelines = [ ActivityTimeline( 0.2, 20 ), ActivityTimeline( 0.2, 20 ), ActivityTimeline( 0.2, 20 ), ActivityTimeline( 0.2, 20 ), ActivityTimeline( 0.2, 20 ) ]
coincidences, coincidenceTimes = FindCoincidences( exampleTimelines, 1 )
TimelinesPlot( exampleTimelines )
CoincidenceBoxes( coincidences, coincidenceTimes, 1 )
mpl.show()

print( coincidences )

In [ ]:
# Large test for performance
largeTimelineSet = []
for i in range( 20 ):
    largeTimelineSet.append( ActivityTimeline( 1, 100 ) )
coincidences, coincidenceTimes = FindCoincidences( largeTimelineSet, 0.1 )
TimelinesPlot( largeTimelineSet )
CoincidenceBoxes( coincidences, coincidenceTimes, 0.1 )

mpl.gcf().set_size_inches(10, 10)
mpl.show()

print( coincidences )

In [ ]:
# Avoid pre-generating timelines since it blows up the RAM
def GenerateCoincidences( DecayRates, EndTime, TimeWindow, TimelinesOut = None ):
    coincidences = []
    coincidenceTimes = []
    
    # Set the first event in each timeline
    nextTimes = []
    for channel in range( len( DecayRates ) ):
        nextTimes.append( DeltaT( DecayRates[ channel ] ) )
        if TimelinesOut is not None:
            TimelinesOut.append( [] )
    
    unfinishedTimeline = True
    while unfinishedTimeline:

        # Check if we have anything to process
        unfinishedTimeline = False
        for nextTime in nextTimes:
            if nextTime <= EndTime:
                unfinishedTimeline = True
                break
        
        # Start the window at the earliest event
        time = min( nextTimes )
        #print( nextTimes, " min = ", time )
    
        # Find all events in the window
        coincidence = []
        for channelIndex, nextTime in enumerate( nextTimes ):
            
            while nextTime <= time + TimeWindow and nextTime <= EndTime:
                
                # Store the time point
                coincidence.append( channelIndex )
                if TimelinesOut is not None:
                    TimelinesOut[ channelIndex ].append( nextTime )
                    
                # Update to next time point
                nextTime += DeltaT( DecayRates[ channelIndex ] )
                
            # Store the first event outside the window, for the next coincidence
            nextTimes[ channelIndex ] = nextTime
        
        # The last entry is empty, because all times now past end
        if len( coincidence ) > 0:
            #print( coincidence )    
            coincidences.append( coincidence )
            coincidenceTimes.append( time )
    
    return coincidences, coincidenceTimes

quickTimelines = []
coincidences, coincidenceTimes = GenerateCoincidences( 20 * [1.0], 10.0, 0.1, quickTimelines )
TimelinesPlot( quickTimelines )
CoincidenceBoxes( coincidences, coincidenceTimes, 0.1 )

mpl.gcf().set_size_inches(10, 10)
mpl.show()

print( coincidences )

In [ ]:
%%time

# Get summaries from a large set
coincidences, coincidenceTimes = GenerateCoincidences( 2000*[60.0], 1.0, 1E-9 )
print( "Number of coincidences: ", len( coincidences ) )
eventsPerCoincidence = 0
for coincidence in coincidences:
    eventsPerCoincidence += len( coincidence )
print( "Average coincidence size: ", eventsPerCoincidence / len( coincidences ) )

In [ ]:
%%time

# Not viable to work with each crystal separately - do whole detector as bulk
import SiemensQuadraProperties as sqp

print( sqp.Lu176decaysInMass( sqp.CrystalMass() ) )
print( sqp.Lu176decaysInMass( sqp.DetectorMass() ) )

# from the paper, 894 MBq of F18, 4.7ns coincidence window
coincidences, coincidenceTimes = GenerateCoincidences( [894E6, sqp.Lu176decaysInMass( sqp.DetectorMass() )], 1E-3, 4.7E-9 )
print( "Number of coincidences: ", len( coincidences ) )
eventsPerCoincidence = 0
crystalEventsPerCoincidence = 0
for coincidence in coincidences:
    eventsPerCoincidence += len( coincidence )
    crystalEventsPerCoincidence += sum( coincidence )
print( "Average coincidence size: ", eventsPerCoincidence / len( coincidences ) )
print( "Average crystal activity per coincidence: ", crystalEventsPerCoincidence / len( coincidences ) )

In [ ]:
from SimulationDataset import *

crystalData = SimulationDataset( "../crystalRadioactivity.log", 1000000, 300.0, 600.0 )
tracerData = SimulationDataset( "../linearF18.log", 1000000, 300.0, 600.0 )

print( crystalData.SampleOneEvent() )

In [ ]:
trueEvents = 0
allEvents = 0

for coincidence in coincidences:
    event = []
    for source in coincidence:
        if source == 0:
            event += tracerData.SampleOneEvent()
        else:
            event += crystalData.SampleOneEvent()
    
    if TwoHitEvent( event ):
        allEvents += 1
        if BackToBackEvent( event, math.pi * 0.05 ):
            trueEvents += 1
            
print( "true events: ", trueEvents )
print( "all events: ", allEvents )
print( "NECR: ", trueEvents * trueEvents / allEvents )